# Prepare the data for analysis
<br>
As data are coming from different sources, datasets must be combined for further analysis

In [1]:
import geopandas as gpd
import pandas as pd

## Ward boundary dataset

In [3]:
# Load raw data
boundariesDistrictBoroughRaw = gpd.read_file("./data/01_Raw/bdline_gpkg_gb/Data/GB/district_borough_unitary_ward_region.shp")
# Filter boundaries data to keep only Essex including Thurrock and Southend-On-Sea
countyList = ['ESSEX_COUNTY','THURROCK_(B)','SOUTHEND-ON-SEA_(B)']
boundariesDistrictBorough = boundariesDistrictBoroughRaw[['NAME','FILE_NAME','AREA_CODE','DESCRIPTIO','CODE','HECTARES','geometry']].copy()
boundariesDistrictBorough['AREA_km2'] = boundariesDistrictBorough['HECTARES']/100
boundariesDistrictBorough = boundariesDistrictBorough[boundariesDistrictBorough['FILE_NAME'].isin(countyList)]
boundariesDistrictBorough.head()

,NAME,FILE_NAME,AREA_CODE,DESCRIPTIO,CODE,HECTARES,geometry,AREA_km2
380,Brizes and Doddinghurst Ward,ESSEX_COUNTY,DIW,District Ward,E05004070,3256.236,"POLYGON ((561084.097 197904.500, 561080.403 19...",32.56236
381,"Ingatestone, Fryerning and Mountnessing Ward",ESSEX_COUNTY,DIW,District Ward,E05004076,2795.113,"POLYGON ((561248.800 198814.899, 561242.699 19...",27.95113
382,Warley Ward,ESSEX_COUNTY,DIW,District Ward,E05004081,1827.160,"POLYGON ((560409.997 187764.202, 560412.297 18...",18.27160
383,"Herongate, Ingrave and West Horndon Ward",ESSEX_COUNTY,DIW,District Ward,E05004071,1984.608,"POLYGON ((560409.997 187764.202, 560407.804 18...",19.84608
384,Shenfield Ward,ESSEX_COUNTY,DIW,District Ward,E05004078,683.749,"POLYGON ((561084.097 197904.500, 561101.296 19...",6.83749


## Population by ward dataset

In [4]:
# Load raw data
populationDistrictBoroughRaw = pd.read_excel("./data/01_Raw/SAPE22DT8a-mid-2019-ward-2019-on-2019 and 2020-LA-syoa-estimates-unformatted.xlsx",
             sheet_name="Mid-2019 Persons", skiprows=4, usecols='A:D,G')

## LSOA boundary dataset

In [2]:
# Load raw data
boundariesLSOARaw = gpd.read_file("./data/01_Raw/LSOA_2011_EW_BFC_shp/LSOA_2011_EW_BFC.shp")
boundariesLSOARaw.head()

,LSOA11CD,LSOA11NM,geometry
0,E01030056,Mid Suffolk 006B,"POLYGON ((602565.167 261619.354, 602568.750 26..."
1,E01030057,Mid Suffolk 007B,"POLYGON ((620353.125 257225.999, 620330.690 25..."
2,E01030054,Mid Suffolk 002B,"POLYGON ((607007.190 275966.413, 607006.813 27..."
3,E01030055,Mid Suffolk 002C,"POLYGON ((607480.834 271499.445, 607477.312 27..."
4,E01030052,Mid Suffolk 001A,"POLYGON ((613156.044 274811.514, 613146.313 27..."


## Population by LSOA

In [3]:
# Load raw data
populationLSOARaw = pd.read_excel("./data/01_Raw/SAPE22DT2-mid-2019-lsoa-syoa-estimates-unformatted.xlsx",
             sheet_name="Mid-2019 Persons", skiprows=4, usecols='A:D,G')

## Train station dataset

In [ ]:
# Load raw data
trainStationRaw = pd.read_csv('./data/01_Raw/table-1410-passenger-entries-and-exits-and-interchanges-by-station.csv',encoding='latin1')
# Select only Essex train station and only some fields
trainStationEssex = trainStationRaw[trainStationRaw['NUTS2 Spatial Unit'] == 'Essex'].copy()
trainStationEssex = trainStationEssex[['Station Name','Region','Local Authority','Constituency','NUTS2 Spatial Unit',
                                       'OS Grid Easting','OS Grid Northing','Station Facility Owner']]
trainStationEssex = trainStationEssex.reset_index(drop=True)
trainStationEssex.head()

In [ ]:
# Convert dataframe to geopandas dataframe
trainStationEssex = gpd.GeoDataFrame(trainStationEssex, 
                       geometry=gpd.points_from_xy(trainStationEssex['OS Grid Easting'],
                                                   trainStationEssex['OS Grid Northing']),
                       crs="epsg:27700")
trainStationEssex.head()

## Merge ward boundary and population datasets for Essex

In [5]:
boundariesPopulationEssex = boundariesDistrictBorough.merge(populationDistrictBoroughRaw,left_on='CODE',right_on="Ward Code 1")
boundariesPopulationEssex = boundariesPopulationEssex.drop(['NAME','Ward Code 1','LA Code (2019 boundaries)'],axis = 1)
boundariesPopulationEssex.head()

,FILE_NAME,AREA_CODE,DESCRIPTIO,CODE,HECTARES,geometry,AREA_km2,Ward Name 1,LA name (2019 boundaries),All Ages
0,ESSEX_COUNTY,DIW,District Ward,E05004070,3256.236,"POLYGON ((561084.097 197904.500, 561080.403 19...",32.56236,Brizes and Doddinghurst,Brentwood,6272.0
1,ESSEX_COUNTY,DIW,District Ward,E05004076,2795.113,"POLYGON ((561248.800 198814.899, 561242.699 19...",27.95113,"Ingatestone, Fryerning and Mountnessing",Brentwood,6260.0
2,ESSEX_COUNTY,DIW,District Ward,E05004081,1827.160,"POLYGON ((560409.997 187764.202, 560412.297 18...",18.27160,Warley,Brentwood,6399.0
3,ESSEX_COUNTY,DIW,District Ward,E05004071,1984.608,"POLYGON ((560409.997 187764.202, 560407.804 18...",19.84608,"Herongate, Ingrave and West Horndon",Brentwood,3696.0
4,ESSEX_COUNTY,DIW,District Ward,E05004078,683.749,"POLYGON ((561084.097 197904.500, 561101.296 19...",6.83749,Shenfield,Brentwood,5400.0


## Merge LSOA boundary with population for Essex

In [6]:
# Need "boundariesPopulationEssex.pkl"
boundariesPopulationEssex = pd.read_pickle("./data/02_Preprocessed/boundariesPopulationEssex.pkl")
boundariesPopulationEssex = boundariesPopulationEssex['LA name (2019 boundaries)'].drop_duplicates()
# Filter population based on "LA name (2019 boundaries)" using merge
populationLSOAEssex = populationLSOARaw.merge(boundariesPopulationEssex,on='LA name (2019 boundaries)')
# Merge population filtered with LSOA boundaries
boundariesPopulationLSOAEssex = boundariesLSOARaw.merge(populationLSOAEssex,left_on='LSOA11CD',right_on='LSOA Code')
boundariesPopulationLSOAEssex = boundariesPopulationLSOAEssex.to_crs('EPSG:27700')
boundariesPopulationLSOAEssex = boundariesPopulationLSOAEssex.drop(['LSOA Code','LSOA Name'], axis=1)
boundariesPopulationLSOAEssex.head()

,LSOA11CD,LSOA11NM,geometry,LA Code (2019 boundaries),LA name (2019 boundaries),All Ages
0,E01022006,Tendring 002C,"POLYGON ((624232.454 231254.095, 624233.006 23...",E07000076,Tendring,1860
1,E01022087,Uttlesford 005F,"POLYGON ((551345.426 224304.625, 551346.745 22...",E07000077,Uttlesford,2898
2,E01022007,Tendring 002D,"POLYGON ((623718.870 231191.886, 623707.172 23...",E07000076,Tendring,1861
3,E01022086,Uttlesford 006A,"POLYGON ((551449.852 224303.744, 551441.643 22...",E07000077,Uttlesford,1762
4,E01022004,Tendring 004D,"MULTIPOLYGON (((623415.943 230599.882, 623418....",E07000076,Tendring,1752


## Save merged dataframe into pickle

In [7]:
boundariesPopulationEssex.to_pickle("./data/02_Preprocessed/boundariesPopulationEssex.pkl")

In [ ]:
trainStationEssex.to_pickle("./data/02_Preprocessed/trainStationEssex.pkl")

In [9]:
boundariesPopulationLSOAEssex.to_pickle("./data/02_Preprocessed/boundariesPopulationLSOAEssex.pkl")